In [0]:
from pyspark.sql.functions import *


In [0]:
df = spark.read.csv("/Volumes/workspace/default/batch12/Orders.csv", header=True, inferSchema=True)
#df.show() # 20 rows
display(df)

In [0]:
#Define schema progarmatically
from pyspark.sql.types import *
orderSchema = StructType([StructField("Region", StringType() ,True)
,StructField("Country", StringType() ,True)
,StructField("ItemType", StringType() ,True)
,StructField("SalesChannel", StringType() ,True)
,StructField("OrderPriority", StringType() ,True)
,StructField("OrderID", IntegerType() ,True)
,StructField("UnitsSold", IntegerType() ,True)
,StructField("UnitPrice", DoubleType() ,True)
,StructField("UnitCost", DoubleType() ,True)
,StructField("TotalRevenue", DoubleType() ,True)
,StructField("TotalCost", DoubleType() ,True)
,StructField("TotalProfit", DoubleType() ,True)
])


df = spark.read.load("/Volumes/workspace/default/batch12/Orders.csv",format="csv", header=True, schema=orderSchema)
display(df)

In [0]:
orderSchema = 'Region String ,Country String ,ItemType String ,SalesChannel String ,OrderPriority String ,OrderID Integer ,UnitsSold Integer ,UnitPrice Double ,UnitCost Double ,TotalRevenue Double ,TotalCost Double ,TotalProfit Double'
                          
df = spark.read.load("/FileStore/tables/Order.csv",format="csv", header=True, schema=orderSchema)
df.printSchema()
#df.schema
#display(df)


In [0]:
from pyspark.sql.functions import *
df1 = df.select("Region", col("Country"), column("SalesChannel"), df["TotalProfit"], df.TotalCost)
display(df1)

# Data frames are IMMUTABLE

In [0]:
df2 = df.select("Region", col("Country").alias("NewCountry"), column("SalesChannel"), df["TotalProfit"], df.TotalCost, expr("TotalProfit*100/TotalCost").alias("ProfitPercent"))
display(df2)

In [0]:
df.createOrReplaceTempView("orders")
df3 = spark.sql("select Region, Country from orders where Country = 'India' order by Region")
display(df3)

In [0]:
%sql
select * from orders;

In [0]:
from pyspark.sql.functions import lit
#Add column
# Rename column
# Delete the column
df4 = df.withColumn("StoreName", lit("Mumbai-Dmart"))
df5 = df.withColumn("ProfitPercent", expr("TotalProfit*100/TotalCost"))
df6 = df.withColumn("ProfitPercent", column("TotalProfit")*100/column("TotalCost"))
df6 = df.withColumn("ProfitPercent", df["TotalProfit"]*100/df.TotalCost)
display(df6)

In [0]:
df7 = df.withColumnRenamed("Country", "New-Country")
display(df7)

In [0]:
df8 = df.drop("Country")
display(df8)

In [0]:
#Where / filter

df9 = df.where("Country = 'India'")
df10 = df.where((col("UnitsSold")>100) & (col("Region")=='Asia'))
df11 = df.filter((col("UnitsSold")>100) & (col("Region")=='Asia'))

display(df11)

In [0]:
df10 = df.orderBy(col("Country").desc())
df11 = df.orderBy(col("Country").desc(), col("Region").asc())
df12 = df.sort(col("Country").desc(), col("Region").asc())
display(df12)

In [0]:
df13 = df.groupBy("Country", "Region").agg(sum("TotalProfit"), max("TotalProfit"), min("TotalProfit"), avg("TotalProfit"), count("TotalProfit"))
display(df13)

In [0]:
# You have to load a orders data. Now find out the following.
# 1. Display all the Country where TotalProit is more than 10000
# 2. Display all the Orders details where Region is Asia and Country is India
# 3. Diaply all the orders detauils where UnitsSold is > 100 and result will be 
# displayed in the decreasing order of the UnitsSold
# 4. Display count of orders generated for every SalesChannel
# 5. Display count of orders and sum of the revenue generated for country India and China separately.

In [0]:
df = spark.read.csv("/Volumes/workspace/default/batch12/Orders.csv", header=True, inferSchema=True)

df.createOrReplaceTempView("orders")
#Display all the Country where TotalProit is more than 10000
#df14 = spark.sql(" select Country from orders where TotalProfit >10000")
df15 = df.where(col("TotalProfit")>10000)
df16 = df15.select("Country")
display(df16)



In [0]:
#Display all the Orders details where Region is Asia and Country is India
# select * from orders where Region = 'Asia' and Country = 'India'
df17 = df.where((col("Region")=='Asia') & (col("Country")=='India'))
display(df17)


In [0]:
# 3. Diaply all the orders detauils where UnitsSold is > 100 and result will be 
# displayed in the decreasing order of the UnitsSold
# 4. Display count of orders generated for every SalesChannel
# 5. Display count of orders and sum of the revenue generated for country India and China separately.

# select * from orders where UnitsSold is > 100 order by UnitsSold
df18 = df.where(col("UnitsSold")>100)
df19 = df18.orderBy(col("UnitsSold").desc())
display(df19)

In [0]:
# 4. Display count of orders generated for every SalesChannel

#select SalesChannel, count(*) from orders
df20 = df.groupBy("SalesChannel").agg(count("*"))
display(df20)

In [0]:
%sql
select SalesChannel, count(*) from orders group by SalesChannel

In [0]:
from pyspark.sql.functions import to_date
df21 = spark.read.csv("/Volumes/workspace/default/batch12/Dates-Batch11.txt", header=True, inferSchema=True)

df22 = df21.withColumn("ConvertToDateType", to_date(col("reg_date"), "dd-MM-yyyy"))
df23 = df22.withColumn("ConvertToTimeStampType", to_timestamp(col("reg_date"), "dd-MM-yyyy"))
df24 = df23.withColumn("TodayDate", current_date())
df25 = df24.withColumn("CurrentTimeStamp", current_timestamp())


df26 = df25.select(year("TodayDate"), month("TodayDate"), dayofmonth("TodayDate"), hour("CurrentTimeStamp"), minute("CurrentTimeStamp"), second("CurrentTimeStamp"))
display(df26)

display(df25)

In [0]:
df27 = df25.select(date_format("TodayDate", "MM/yyyy"))
display(df27)



In [0]:
df28 = df27.distinct()
df28.collect()

In [0]:
df29 = spark.read.csv("/Volumes/workspace/default/batch12/null.txt", header=True, inferSchema=True)
df30 = df29.na.drop() #any
df31 = df29.na.drop("all")
df32 = df29.na.drop("all", subset=["id", "city"])
df33 = df29.dropna()
display(df33)

In [0]:
df34 = df29.na.fill("Not Available")
data = {"id":0, "name":"NA", "city":"Unknown"}
df35 = df29.na.fill(data)
display(df35)

In [0]:
df36= spark.read.json("/Volumes/workspace/default/batch12/SalesLT.Customer.json")
display(df36)

In [0]:
df40 = spark.read.parquet("/Volumes/workspace/default/batch12/SalesLT.Customer.parquet")
display(df40)

In [0]:
df = spark.read.csv("fsdbfdsmnbfds", sep='|')

In [0]:
df42 = spark.read.csv("/Volumes/workspace/default/batch12/Multiline_example.csv", header=True, inferSchema=True, multiLine=True)
display(df42)

In [0]:
df43 = spark.read.json("/Volumes/workspace/default/batch12/Multiline_example.json", multiLine=True)
display(df43)

In [0]:
df44 = spark.read.json("/Volumes/workspace/default/batch12/SampleNested.json")
#display(df44)
df45 = df44.select("age", "email","name", "address.city","address.street", "address.zip", "address.state")
display(df45)


In [0]:
from pyspark.sql.functions import col, explode,collect_list
df46 = spark.read.json("/Volumes/workspace/default/batch12/Array_Explode_Example.json", multiLine=True)
display(df46)

df47 = df46.select("customer_id", col("product_ids")[0], col("product_ids")[1], col("product_ids")[2])
df48 = df46.select("customer_id", explode("product_ids").alias("product_ids"))
df49 = df48.groupby("customer_id").agg(collect_list("product_ids").alias("product_ids"))

display(df49)

In [0]:
df50 = spark.read.csv("/Volumes/workspace/default/batch12/duplicate.txt", inferSchema=True, header=True)
display(df50)
df51 = df50.distinct()
df52 = df50.dropDuplicates()
df53 = df50.dropDuplicates(subset=["city", "name"])
display(df53)


In [0]:
df = spark.read.csv("/Volumes/workspace/default/batch12/Orders.csv", header=True, inferSchema=True)

display(df)

In [0]:
from pyspark.sql.functions import *
df54 = df.select(lower("Region"), upper("Region"), initcap("Region"), ltrim("Region"), rtrim("Region"), trim("Region"), length("Region"), translate("Region", "A", "B"), concat_ws("||", "Region", "Country"), substring("Region", 1, 2), substring_index("Region", " ", 1), split("Region", " "), split("Region", " "))
display(df54)

In [0]:
df55 = spark.read.csv("/Volumes/workspace/default/batch12/batch11_wrong_date.txt", header=True, inferSchema=True)
display(df55)
df56 = df55.withColumn("OrderDate", try_to_date(col("reg_date"), "dd/MM/yy"))
display(df56)


In [0]:
#df55.printSchema()
result = df55.collect()
print(result[0][1])
if result[0][1] =='Deepak':
    print("Thank you")

# YOU will never USE THE COLLECT Function

In [0]:
df57 = df55.limit(2)
df57 = df55.first()
print(df57[1])


In [0]:
df55 = spark.read.csv("/Volumes/workspace/default/batch12/batch11_wrong_date.txt", header=True, inferSchema=True)
display(df55)
df56 = df55.withColumn("OrderDate", try_to_date(col("reg_date"), "dd/MM/yy"))
display(df56)

In [0]:
#WRITING THE DATAFRAME
# overwrite
# append
# ignore
# error
df56 = df56.repartition(4)
df56.write.mode("error").format("json").option("path", "/Volumes/workspace/default/batch12/Output/batch12_correct_date_example3").save()

In [0]:
df56.rdd.getNumPartition()

In [0]:
df59 = spark.read.json("/Volumes/workspace/default/batch12/Output/batch12_correct_date_example/part-00000-tid-4408820544819278575-7fca9d4f-9d27-40e8-b3b6-9857852553f0-379-1-c000.json")

display(df59)